# Implement OAuth Authentication

> How to add OAuth authentication to your Plash app

This guide shows you how to add OAuth authentication to your Plash app. We will be using Google OAuth where users can log in with their existing Google accounts.

## Step by step tutorial

In this tutorial, you'll learn how to use environment variables in your Plash app.

**Prerequisites:**

* A registered account at https://pla.sh
* The Plash CLI installed (run `pip install plash-cli` if needed)
* Logged in via the CLI (run `plash_login` if needed)

## Setup

In [ ]:
#| hide
from plash_cli._bash_magic import *
import tempfile
temp_dir = tempfile.TemporaryDirectory()
td = temp_dir.name

In [ ]:
%%bash
#| hide
cd $`td`

## Step 1: Set up Google OAuth credentials

Create OAuth credentials in the Google Cloud Console:

1. Go to [Google Cloud Console](https://console.cloud.google.com/)
2. Create a new project or select an existing one
3. Go to "APIs & Services" > "Credentials"
4. Click "Create Credentials" > "OAuth client ID"
5. Select "Web application" as the application type
6. Add `https://your-app-id.pla.sh/redirect` to the authorized redirect URIs
7. Save your Client ID and Client Secret

## Step 2: Create a project directory

In [ ]:
%%bash
mkdir -p oauth-auth-app
cd oauth-auth-app

In [ ]:
%%writefile .plash
export PLASH_APP_ID='oauth-example-app'

## Step 3: Create the environment variables file

In [ ]:
%%writefile plash.env
export GOOGLE_CLIENT_ID="your-google-client-id"
export GOOGLE_CLIENT_SECRET="your-google-client-secret"

## Step 4: Create the main application file

In [ ]:
%%writefile main.py
from fasthtml.common import *
from fasthtml.oauth import OAuth, GoogleAppClient
import os

# Initialize the app
app, rt = fast_app()

# Create Google OAuth client
client = GoogleAppClient(
    os.environ.get("GOOGLE_CLIENT_ID"),
    os.environ.get("GOOGLE_CLIENT_SECRET")
)

# Define authentication class
class Auth(OAuth):
    def get_auth(self, info, ident, session, state):
        email = info.email or ''
        if info.email_verified:
            session['user'] = {'name': info.name}
            return RedirectResponse('/', status_code=303)

# Initialize OAuth
oauth = Auth(app, client)

# Home route
@rt
def index(req, sess):
    # Check if user is logged in
    user = sess.get("user") if sess else None
    
    style = "font-family: system-ui; max-width: 600px; margin: 0 auto; padding: 2rem;"
    
    if user:
        # Logged in view
        return Div(
            H1(f"Hello, {user['name']}!"),
            P("You are logged in with Google."),
            A("Logout", href="/logout"),
        )
    else:
        # Login page
        return Div(
            H1("OAuth Example"),
            P("Click below to log in:"),
            A("Login with Google", href=oauth.login_link(req)),
        )

# Login route
@rt
def login(req):
    style = "font-family: system-ui; max-width: 600px; margin: 0 auto; padding: 2rem;"
    return Div(
        H1("Login"),
        A("Login with Google", href=oauth.login_link(req)),
        style=style
    )

serve()

## Step 5: Create the requirements file

In [ ]:
%%writefile requirements.txt
python-fasthtml>=0.7.0

## Step 6: Deploy your application

In [ ]:
%%bash
#| output: false
plash_deploy

Initializing deployment...
✅ Upload complete! Your app is currently being built.
It will be live at https://oauth-example-app.pla.sh



That's it! Your app now has Google OAuth authentication. When users visit, they'll see a login button that redirects them to Google for authentication.

::: {.callout-tip}
For more OAuth features, check out the [FastHTML OAuth documentation](https://fastht.ml/docs/explains/oauth.html).
:::

**🎉 Complete: Environment Variables Tutorial**